<a href="https://colab.research.google.com/github/peterlhn/Google-Colab/blob/master/Matrix_Practices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm
import pandas as pd

In [ ]:
class case:
  def __init__(self, id, mean, sd, pub_p, pub_v, retailer_p, retailer_s):
    self.mean = mean
    self.sd = sd
    self.pub_p = pub_p
    self.pub_v = pub_v
    self. retailer_p = retailer_p
    self.retailer_s = retailer_s
  

In [ ]:
case_low = case(1, 20000, 4000, 5, 1, 24, 3)
case_mid = case(2, 20000, 4000, 10, 1, 24, 3)
case_high = case(3, 20000, 4000, 20, 1, 24, 3)

list_case = ['case_low', 'case_mid', 'case_high']

print(case_low.pub_p)

for i in list_case:
  print(i)


5
case_low
case_mid
case_high


In [ ]:
data = []

for i in range(0,100,5):
  x = best_revenue_share(i/100) 
  data.append(x)


In [ ]:
df = pd.DataFrame(data, columns=['Revenue Share','Cost of Understock','Cost of Overstock','Critical Fractile', 'Optimal Order Quantity', 'Actual Sales', 'Understock', 'Overstock', 'Retailers Profit', 'Publisher Profit', 'Supplychain Profit']).round(2)

In [ ]:
df

,Revenue Share,Cost of Understock,Cost of Overstock,Critical Fractile,Optimal Order Quantity,Actual Sales,Understock,Overstock,Retailers Profit,Publisher Profit,Supplychain Profit
0,0.00,14.5,6.5,0.69,21988.80,260384.0,19205.35,794.65,2783.45,186905.0,447289.0
1,0.05,13.3,6.5,0.67,21778.64,237378.0,19138.34,861.66,2640.30,208084.0,445462.0
2,0.10,12.1,6.5,0.65,21547.09,214458.0,19059.88,940.12,2487.21,228894.0,443352.0
3,0.15,10.9,6.5,0.63,21289.72,191641.0,18966.85,1033.15,2322.87,249243.0,440884.0
4,0.20,9.7,6.5,0.60,21000.61,168944.0,18854.87,1145.13,2145.74,269008.0,437952.0
5,0.25,8.5,6.5,0.57,20671.58,146398.0,18717.58,1282.42,1954.00,288013.0,434411.0
6,0.30,7.3,6.5,0.53,20290.88,124037.0,18545.45,1454.55,1745.43,305999.0,430036.0
7,0.35,6.1,6.5,0.48,19840.81,101909.0,18323.37,1676.63,1517.44,322563.0,424472.0
8,0.40,4.9,6.5,0.43,19292.72,80090.0,18025.71,1974.29,1267.01,337035.0,417125.0
9,0.45,3.7,6.5,0.36,18595.48,58697.0,17604.59,2395.41,990.88,348192.0,406889.0


In [ ]:
# Method - Find Revenue Sharing affecting Supply Chain Profit
def best_revenue_share(f):
    mean = 20000
    sd = 4000
    f = f

    pub_p = 9.5
    pub_v = 1
    retailer_p = 24
    retailer_c = pub_p
    retailer_s = 3

    # ------ Cost of Understocking / Overstocking ------- #
    cu = (1-f)*retailer_p - retailer_c
    co = retailer_c - retailer_s
    # Critical Fractile = Cu/(Cu+Co)
    cf = cu/(cu+co)
    # Optimal Order Quantity (Q*) 
    Q = norm.ppf(cf, loc=mean, scale=sd)

    # -------  Number of Under / Overstocking -------- # 
    num_diff = sd *  norm(loc = 0 , scale = 1).pdf((Q-mean)/sd)

    # Understocking # 
    e_under = 1 - norm(loc = mean , scale = sd).cdf(Q)
    num_under_ind = (mean - Q)*e_under 
    num_under_sum = num_diff + num_under_ind
    
    # Overstocking # 
    e_over = norm(loc = mean , scale = sd).cdf(Q)
    num_over_ind =  (Q-mean)*e_over
    num_over_sum = num_diff + num_over_ind

    # Actual Sales #
    act_sale = Q - num_over_sum

    # ------- Retailer Profit ---------- #
    # (1-f)*price * Actual Sales #
    sales_retailer = round((1-f)*retailer_p * act_sale,0)

    # Overstock # X Salvage value
    overstock_sales_retailer = round(num_over_sum * retailer_s,0)

    # Optimal Order Quantity * Cost
    cost_retailer = round(retailer_c * Q,0)

    # Profit = Sum of 3 above 
    profit_retailer = sales_retailer + overstock_sales_retailer - cost_retailer

    # ------ Publisher Profit -------- #
    # price * Q
    sales_publisher = round(pub_p * Q,0)

    # Revenue Share = f * p * act_sale
    share_revenue_publisher = round(f * retailer_p * act_sale,0)

    # Optimal Order Quantity * Cost
    cost_publisher = round(pub_v * Q,0)

    # Profit = Sum of 3 above 
    profit_publisher = sales_publisher + share_revenue_publisher - cost_publisher

    # ------ Supplychain Profit -------- #
    supply_chain_profit = profit_publisher + profit_retailer

    return [f, cu, co, cf, Q, profit_retailer, act_sale,num_under_sum, num_over_sum, profit_publisher, supply_chain_profit]


In [ ]:
r,p,s = best_revenue_share(0.1)
print(f'{r}, {p}, {s}')

Cost of Understocking (Cu) is 12.1
Cost of Overstocking (Co) is 6.5
Optimal Order Quantity is 21547.0, given Critical Fractile is 0.6505376344
Understock is -541.0 + 1481.0 = 940.0, given the standard index is 0.35
Overstock is 1006.0 + 1481.0 = 2487.0, given the standard index is 0.65
Actual Sales at P is 19060.0
411693.0, 7462.0, 204697.0
Retailers profit is 214458.0
204697.0, 45744.0, 21547.0
Retailers profit is 228894.0
Supply Chain Profit is 443352.0
214458.0, 228894.0, 443352.0


###**Single Calculation**

In [ ]:
import numpy as np

**Model Inputs**

In [ ]:
mean = 20000
sd = 4000

# Publisher
pub_p = 9.5
pub_v = 1

# Retailer
f = 0.1      # # Revenue Sharing %
retailer_p = 24
retailer_c = pub_p
retailer_s = 3

**Critical Numbers**

In [ ]:
# Cost of Understocking, Cu = (1-f)*p - c
cu = (1-f)*retailer_p - retailer_c
# Cost of Overstocking, Co = c - s
co = retailer_c - retailer_s

# Critical Fractile = Cu/(Cu+Co)
# Chance of successfuly meet the demand 
cf = cu/(cu+co)

# Optimal Order Quantity (Q*) 
# NORM.INV in Excel -> loc = mean, scale = sd
Q = norm.ppf(cf, loc=mean, scale=sd)


print(f"Cost of Understocking (Cu) is {round(cu,1)}")
print(f"Cost of Overstocking (Co) is {round(co,1)}")

print(f"Optimal Order Quantity is {round(Q,0)}, given Critical Fractile is {round(cf,10)}")

Cost of Understocking (Cu) is 12.1
Cost of Overstocking (Co) is 6.5
Optimal Order Quantity is 21547.0, given Critical Fractile is 0.6505376344


**Expected Number of Over & Understock**

In [ ]:
# Probability that the demand will be under / over Optimal Order Quantity
# cdf = cumulative distribution function.
num_diff = sd *  norm(loc = 0 , scale = 1).pdf((Q-mean)/sd)

# NORM.DIST(20000, 4000, 21547,1) = 0.65 =CF
# pdf = Probability density function.
# num_under = (mean-sd)*(1-NORM.DIST(Q*,mean,sd,1))
e_under = 1 - norm(loc = mean , scale = sd).cdf(Q)
num_under_ind = (mean - Q)*e_under 
num_under_sum = num_diff + num_under_ind
print(f'Understock is {round(num_under_ind,0)} + {round(num_diff,0)} = {round(num_under_sum,0)}, given the standard index is {round(e_under,2)}')


# num_under = (mean-sd)*(1-NORM.DIST(Q*,mean,sd,1))
e_over = norm(loc = mean , scale = sd).cdf(Q)
num_over_ind =  (Q-mean)*e_over
num_over_sum = num_diff + num_over_ind

print(f'Overstock is {round(num_over_ind,0)} + {round(num_diff,0)} = {round(num_over_sum,0)}, given the standard index is {round(e_over,2)}')

# Actual Sales = Q* - Overstock
act_sale = Q - num_over_sum
print(f'Actual Sales at P is {round(act_sale,0)}')

Understock is -541.0 + 1481.0 = 940.0, given the standard index is 0.35
Overstock is 1006.0 + 1481.0 = 2487.0, given the standard index is 0.65
Actual Sales at P is 19060.0


**Revenue, Cost, and Profit**

**Retailer**

In [ ]:
# (1-f)*price * Actual Sales #
sales_retailer = round((1-f)*retailer_p * act_sale,0)

# Overstock # X Salvage value
overstock_sales_retailer = round(num_over_sum * retailer_s,0)

# Optimal Order Quantity * Cost
cost_retailer = round(retailer_c * Q,0)

# Profit = Sum of 3 above 
profit_retailer = sales_retailer + overstock_sales_retailer - cost_retailer

print(f'{sales_retailer}, {overstock_sales_retailer}, {cost_retailer}')
print(f'Retailers profit is {profit_retailer}')

411693.0, 7462.0, 204697.0
Retailers profit is 214458.0


**Manufacturer**

In [ ]:
# price * Q
sales_publisher = round(pub_p * Q,0)

# Revenue Share = f * p * act_sale
share_revenue_publisher = round(f * retailer_p * act_sale,0)

# Optimal Order Quantity * Cost
cost_publisher = round(pub_v * Q,0)

# Profit = Sum of 3 above 
profit_publisher = sales_publisher + share_revenue_publisher - cost_publisher

print(f'{sales_publisher}, {share_revenue_publisher}, {cost_publisher}')
print(f'Retailers profit is {profit_publisher}')

204697.0, 45744.0, 21547.0
Retailers profit is 228894.0


###**Reference**